## Preliminary analysis

This notebook analyzes the MTA subway data for the week of June 10-17, 2017 that can be found here:

http://web.mta.info/developers/turnstile.html

1.Download that SAME file and read it in below. View the first few rows.

In [1]:
import pandas as pd

file = pd.read_csv('http://web.mta.info/developers/data/nyct/turnstile/turnstile_170617.txt')

URLError: <urlopen error [Errno 110] Connection timed out>

2.What are the column names?

In [0]:
#insert 2
file.columns

3.We can see that there is a lot of whitespace at the end of the exits column name. Let's strip that whitespace:

In [0]:
#insert 3
file.columns = file.columns.str.strip()
file.columns

4.How big is the data set?

In [0]:
file.shape

5.How many unique stations are there? What are they? Answer each of these questions in one line each.

In [0]:
#insert 5
# file.groupby(['STATION', 'C/A', 'UNIT', 'SCP']).count()
print(len(set(file.STATION)))
print('The stations are:', set(file.STATION))

6.Okay, so we understand what station represents. But what the heck are C/A, UNIT, and SCP? Keep in mind that in the larger stations, you might have multiple areas within one station that look like this:

<img src="image.jpg" style="width: 300px;"/>

Further complicating things, there are a few station names like 14TH ST that refer to more than one station location along that street.

This data set is not very well documented. Welcome to the joys of real world data science!!!

Read the following two links carefully to see other people's confusion and what information they have been able to gather:

https://groups.google.com/forum/#!topic/mtadeveloperresources/AMVx2WUY9iI

https://groups.google.com/forum/#!searchin/mtadeveloperresources/%22remote$20unit%22%7Csort:relevance/mtadeveloperresources/z8l3ZU9cY6Y/OFlHGkFAimQJ

It sounds like each C/A + UNIT + SCP + STATION combo refers to a single turnstile. How many unique turnstiles are there? 

In [0]:
#insert 6
print(len(file.groupby(['STATION', 'C/A', 'UNIT', 'SCP']).count()))

7.What data types are each of the columns?

In [0]:
#insert 7
file.info()

8.We can see that the exits and entries are treated as integers but the others are all treated as objects (strings). Overwrite the time column so that it is a datetime object containing the combined date and time column info (so that the times have a chronological order). 

In [0]:
#insert 8
file.TIME = pd.to_datetime(file.DATE + ' ' + file.TIME, format = '%m/%d/%Y %H:%M:%S')
del file['DATE']
file.info()

9.What is the earliest and latest date in our dataset?

In [0]:
#insert 9
print(file.TIME.min())
print(file.TIME.max())

10.If we wanted to only look at the 34st Street Penn Station stop on 6/12/2017, what would we type?

In [0]:
#insert 10
import datetime
date = datetime.date(year=2017, month=6, day=12)
file[(file['STATION'] == '34 ST-PENN STA') & (file['TIME'] == date)]

11.Create a dictionary called bigDict. It should contain a nested set of keys and values. The outermost key should be the tuple (C/A,UNIT,STATION) and its value should itself be a dictionary with the SCP as the key and a list of (TIME, EXITS) tuples as its values. The purpose of this section is to prepare data for later uses. It should take a little while to finish running.

In [0]:
bigDict = {}
for i,row in file.iterrows():
    if (row['C/A'], row['UNIT'], row['STATION']) in bigDict:
        if row['SCP'] in bigDict[(row['C/A'], row['UNIT'], row['STATION'])]:
            bigDict[(row['C/A'], row['UNIT'], row['STATION'])][row['SCP']].append((row['datetime'], row['EXITS']))
        else:
            bigDict[(row['C/A'], row['UNIT'], row['STATION'])][row['SCP']] = [(row['datetime'], row['EXITS'])]
    else:
        bigDict[(row['C/A'], row['UNIT'], row['STATION'])] = {row['SCP'] : [(row['datetime'], row['EXITS'])]}

12.As an example, use the bigDict to view all of the turnstile data located at the('A037', 'R170', '14 ST-UNION SQ') area:

In [0]:
#insert 12
print(bigDict[('A037', 'R170', '14 ST-UNION SQ')])

13.Create a function called inspection that takes in the (C/A,UNIT,STATION) tuple and SCP value and plots the exit counter data versus time. 

For example, the input of 
```python
inspection(('A037', 'R170', '14 ST-UNION SQ'), '05-00-00')
```
should produce an upward trending plot.

In [0]:
#insert 13
import matplotlib.pyplot as plt

def inspection(bigDict_tuple, SCP):
    times = []
    exits = []
    for time, count in bigDict[bigDict_tuple][SCP]:
        times.append(time)
        exits.append(count)
    plt.plot(times, exits)
    plt.xlabel('Timestamp')
    plt.ylabel('Exit Count')
    plt.title('People exited at a given time')

inspection(('A037', 'R170', '14 ST-UNION SQ'), '05-00-00')

## Finding Data Errors
14.Due to bugs in MTA data, we will need to remove "incorrect" data. First, find the incorrect data by figuring out which turnstile counters aren't going strictly upwards. How many of these incorrect data values are there? Create a smaller dictionary callled "trouble" that contains the troublesome data from the bigDict.

In [0]:
#insert 14
trouble = {}
for st,stv in bigDict.items():
    for scp,lst in stv.items():
        #Cleaning in Each LIST of turnstile
        toDel = []
        n = len(lst)
        lst.sort()
        for i in range(1,n-1):
            if(lst[i-1][1]<=lst[i][1] and lst[i][1]<=lst[i+1][1]): #What we expected Data to be (Non-Decreasing)
                continue
            key = (st,scp)
            trouble[key] = trouble.get(key,0)+1
print("Trouble List: ",len(trouble.keys()))
for k,v in trouble.items():
    print(k,v)

15.Using the troublesome dictionary and your inspection plotting function, plot all of the troublesome data. There are several different types of errors. What do you think is causing each type?

In [0]:
#insert 15
for station, SCP_dict in trouble.items():
    inspection(station[0], station[1])

## Data Cleanup
There are three types of mistakes: decreasing, garbage values, and turnstile resets.

#### Mistake Type I: Monotone but Decreasing - To fix this, we reflect the data. 

16.Run the cell below to fix it:

In [0]:
def isMonotoneDecrease(tup):
    '''Input: Tuple of (Station,SCP). 
    Output: True if this SCP has monotone property, but decreasing, False otherwise.'''
    n = len(bigDict[tup[0]][tup[1]])
    for i in range(n-1):
        if(bigDict[tup[0]][tup[1]][i+1][1]>bigDict[tup[0]][tup[1]][i][1]):
            return False
    return True

def fixMonotoneDecrease(tup):
    '''reflects the data to fix it'''
    n = len(bigDict[tup[0]][tup[1]])
    for i in range(n):
        bigDict[tup[0]][tup[1]][i] = (bigDict[tup[0]][tup[1]][i][0],(-1)*bigDict[tup[0]][tup[1]][i][1])
    

monotoneDecreaseList = []
for k in trouble:
    if(isMonotoneDecrease(k)):
        monotoneDecreaseList.append(k)
print("Total Monotone Decrease:",len(monotoneDecreaseList))
for k in monotoneDecreaseList:
    fixMonotoneDecrease(k)
print("Problem Fixed!")

for k in trouble:
    if(isMonotoneDecrease(k)):
        monotoneDecreaseList.append(k)
print("Total Monotone Decrease:",len(monotoneDecreaseList))
for k in monotoneDecreaseList:
    inspection(k[0], k[1])

#### Mistake Type II: Garbage Value - To fix this, remove the garbage value

17.Run the cell below to fix it:

In [0]:
def garbageEliminator(tup):
    '''removes nonsensical isolated points'''
    n = len(bigDict[tup[0]][tup[1]])
    toDel = []
    for i in range(1,n-1):
        if((bigDict[tup[0]][tup[1]][i-1][1]>bigDict[tup[0]][tup[1]][i+1][1])):
            continue
        if((bigDict[tup[0]][tup[1]][i-1][1]<=bigDict[tup[0]][tup[1]][i][1]) and (bigDict[tup[0]][tup[1]][i][1]<=bigDict[tup[0]][tup[1]][i+1][1])):
            continue
        toDel.append(bigDict[tup[0]][tup[1]][i])
    #Deletion Process
    if(len(toDel)==0):
        return 0
    for k in toDel:
        bigDict[tup[0]][tup[1]].remove(k)
    return 1


#Driver
cnt = 0
healList = []
for k in trouble:
    if(garbageEliminator(k)):
        healList.append(k)
print("Garbage Removed:",len(healList))
for k in healList:
    inspection(k[0], k[1])

#### Mistake Type III: Turnstile Reset - To fix this, shift the data upwards.

18.Run the cell below to fix it:

In [0]:
def dealingWithReset(tup):
    '''for counters that are reset, we fix the data by shifting it upwards'''
    sta = tup[0]
    tsl = tup[1]
    n = len(bigDict[sta][tsl])
    #Detecting Part
    resetPoint = [] # it means (i,i+1) is reset
    resetSet = []
    for i in range(1,n-2):
        if(bigDict[sta][tsl][i][1]<=bigDict[sta][tsl][i+1][1]):
            continue #We don't need to change this one
        resetPoint.append(i)
    #Fixing Part
    resetSet = set(resetPoint)
    cumulative = 0
    for i in range(n-2):
        if(i not in resetSet):
            bigDict[sta][tsl][i] = (bigDict[sta][tsl][i][0],bigDict[sta][tsl][i][1]+cumulative)
            continue
        #Problem
        expected = (bigDict[sta][tsl][i][1]-bigDict[sta][tsl][i-1][1])+ (bigDict[sta][tsl][i+2][1]-bigDict[sta][tsl][i+1][1])
        expected = int(expected/2)
        shift = (bigDict[sta][tsl][i][1]+expected)-bigDict[sta][tsl][i+1][1]
        cumulative = shift
    for i in range(n-2,n):
        bigDict[sta][tsl][i] = (bigDict[sta][tsl][i][0],bigDict[sta][tsl][i][1]+cumulative)
    #Done!
    
#Test Usage
inspection(('JFK03', 'R536', 'JFK JAMAICA CT1'), '00-03-00')
dealingWithReset((('JFK03', 'R536', 'JFK JAMAICA CT1'), '00-03-00'))
print("Cleaned")
inspection(('JFK03', 'R536', 'JFK JAMAICA CT1'), '00-03-00')

## Overall Cleaning Process
19.This next cell does all of the previous cleanup in one cell. Run the cell below:

In [0]:
toClean = {}
for st,stv in bigDict.items():
    for scp,lst in stv.items():
        #Cleaning in Each LIST of turnstile
        toDel = []
        n = len(lst)
        lst.sort()
        for i in range(1,n-1):
            if(lst[i-1][1]<=lst[i][1] and lst[i][1]<=lst[i+1][1]): #What we expected Data to be (Non-Decreasing)
                continue

            key = (st,scp)
            toClean[key] = trouble.get(key,0)+1
            
for k in toClean.keys():
    if(isMonotoneDecrease(k)):
        fixMonotoneDecrease(k)
    garbageEliminator(k)
    dealingWithReset(k)
    inspection(k[0], k[1])

20.Which troublesome stations are left?

In [0]:
trouble = {}
for st,stv in bigDict.items():
    for scp,lst in stv.items():
        #Cleaning in Each LIST of turnstile
        toDel = []
        n = len(lst)
        lst.sort()
        for i in range(1,n-1):
            if(lst[i-1][1]<=lst[i][1] and lst[i][1]<=lst[i+1][1]): #What we expected Data to be (Non-Decreasing)
                continue
                
            key = (st,scp)
            trouble[key] = trouble.get(key,0)+1
print("Trouble List: ",len(trouble.keys()))
for k,v in trouble.items():
    print(k,v)

21.Delete these two keys from bigDict manually:

In [0]:
#insert 21
del bigDict[('PTH11', 'R545', '14TH STREET')]['00-00-03']
del bigDict[('R240', 'R047', 'GRD CNTRL-42 ST')]['00-00-01']

22.The data is now all cleaned, so let's save it so that we don't have to run all of the above code every time. Use the pickle package to save bigDict as an "MTAdict.pkl" file.

In [0]:
#insert 22
import pickle
with open('MTAdict.pkl', 'wb') as file:
    pickle.dump(bigDict, file)

# --At this point, the data is ready to use and so we are ready for data analysis.--
23.Let's read the cleaned data file back in and save it as bigDict.

In [0]:
#insert 23
file = open('MTAdict.pkl', 'rb')
bigDict = pickle.load(file)

24.Create a function called turnstileRiders that takes in a single turnstile's date/exit info and a start and endtime (in datetime format) and returns the number of riders through that turnstile within that time period. As an extension, you may want to use a linear approximation in the case of incomplete information. 

For instance, the input 
```python
t1 = dt.strptime("2017-06-12 00:00:00","%Y-%m-%d %H:%M:%S")
t2 = dt.strptime("2017-06-13 00:00:00","%Y-%m-%d %H:%M:%S")
turnstileRiders(bigDict[('R204', 'R043', 'WALL ST')]["02-00-00"],t1,t2)
```

should output 1419 riders.

In [0]:
#insert 24
from datetime import datetime
def turnstileRiders(station, t1, t2):
    if t1 < station[0][0]:
        count1 = station[0][1]
    for time, count in station:
        if t1 >= time:
            count1 = count
        if t2 >= time:
            count2 = count
    return count2 - count1

t1 = datetime.strptime("2017-06-12 00:00:00","%Y-%m-%d %H:%M:%S")
t2 = datetime.strptime("2017-06-13 00:00:00","%Y-%m-%d %H:%M:%S")
turnstileRiders(bigDict[('R204', 'R043', 'WALL ST')]["02-00-00"],t1,t2)

25.Create a function called stationRiders that calls the turnstileRiders function for each turnstile in a  (C/A,UNIT,STATION) station area and tallies all of the riders through that area between two times.

For example, an input of 
```python
getStationRangeRider(bigDict[('R204', 'R043', 'WALL ST')],dt(2017,6,12,0,0,0),dt(2017,6,13,0,0,0))
```
should output 9507 riders.

In [0]:
#insert 25
from datetime import datetime as dt
def getStationRangeRider(station, t1, t2):
    total = 0
    for scp, times_list in station.items():
        total += turnstileRiders(times_list, t1, t2)
    return total

getStationRangeRider(bigDict[('R204', 'R043', 'WALL ST')],dt(2017,6,12,0,0,0),dt(2017,6,13,0,0,0))

26.There are still several station areas within a station. Make a plot of the day of the week versus the number of total station rider exits for the ENTIRE Wall St station.

In [0]:
#insert 26
import datetime as dt
t1 = dt.datetime(2017,6,9,0,0,0)
day_count = {}
for station in bigDict:
    if 'WALL ST' == station[2]:
#         print(station)
        for i in range(7):
            t1 += dt.timedelta(days = 1)
            t2 = t1 + dt.timedelta(days = 1)
#             print(t1, t2)
            day = t1.strftime('%a')
            if day in day_count:
                day_count[day] += getStationRangeRider(bigDict[station],t1,t2)
            else:
                day_count[day] = getStationRangeRider(bigDict[station],t1,t2)
print(day_count)
days = []
for day, count in day_count.items():
    print(day)
    for person in range(count):
        days.append(day)
plt.hist(days, bins = np.arange(-0.5, 6.6, 1), rwidth = 0.8)
plt.xlabel('Day of the week')
plt.ylabel('Number of people')
plt.title('Number of people per day on Wall St')

27.Sort by busiest station areas during 6/12 midnight - 6/13 midnight in descending order by creating a list of sorted tuples:

In [0]:
#insert 27
countsstations = []
t1 = dt.datetime(2017,6,12,0,0,0)
t2 = t1 + dt.timedelta(days = 1)
for station in bigDict:
    try:
        countsstations.append((getStationRangeRider(bigDict[station],t1, t2), station))
    except:
        print(station)
print(sorted(countsstations, reverse = True)[:10])

28.Make a dictionary called total_dict that contains the station name as its key and the total number of riders through all of its station areas between 6/12-6/13 as its value. Then create a sorted list of tuples to view the busiest stations on that day.

In [0]:
#insert 28
stations_counts = {}
t1 = dt.datetime(2017,6,12,0,0,0)
t2 = t1 + dt.timedelta(days = 1)
for station in bigDict:
    try:
        if station[2] in stations_counts:
            stations_counts[station[2]] += getStationRangeRider(bigDict[station],t1, t2)
        else:
            stations_counts[station[2]] = getStationRangeRider(bigDict[station],t1, t2)
    except:
        print(station)
counts_stations = []
for station, count in stations_counts.items():
    counts_stations.append((count, station))
print(sorted(counts_stations, reverse = True)[:10])

29.Make a histogram of those station totals:

In [0]:
#insert 29
totals = []
counts_stations.sort(reverse = True)
for count, station in counts_stations[:5]:
    for each_one in range(count):
        totals.append(station)
plt.hist(totals, bins = np.arange(-0.5, 5.6, 1), rwidth = 0.5)
plt.xticks(fontsize = 6)
plt.yticks(fontsize = 8)
plt.xlabel('Station')
plt.ylabel('People')
plt.title('The most populous stations on 6/12/17')
plt.show()

30.Create a commuter index to be the average weekday exits divided by the sum of the avg weekday exits + avg weekend exits. To do this, first make a function called isWeekday that returns True if the datetime input is a weekday and False if it isn't.

In [0]:
#insert 31
def isWeekday(time):
    if time.weekday() <= 4:
        return True
    return False

31.Make a function called commuterIndex that inputs a (C/A,UNIT,STATION) tuple and outputs its commuter index. 

For example, the output of 
```python
getCommuteIndex(('PTH11', 'R545', '14TH STREET'))
```
should be 0.663.

In [0]:
#insert 31
def getCommuteIndex(station):
    t1 = dt.datetime(2017,6,9,0,0,0)
    weekday_exits = 0
    weekend_exits = 0
    for i in range(7):
        t1 += dt.timedelta(days = 1)
        t2 = t1 + dt.timedelta(days = 1)
#         print(t1, t2, isWeekday(t1))
        if isWeekday(t1):
#             print('adding to weekday_exits')
            weekday_exits += getStationRangeRider(bigDict[station], t1, t2)
        else:
            weekend_exits += getStationRangeRider(bigDict[station], t1, t2)
    return weekday_exits / (weekday_exits + weekend_exits)

32.Create a sorted list of tuples in descending order containing the commuter index and the station area tuple.

In [0]:
#insert 32
tuple_list = []
for station in bigDict:
    try:
        tuple_list.append((getCommuteIndex(station), station))
    except:
        print(station, 'is a problem')
tuple_list.sort(reverse = True)
print(tuple_list[:10])

33.Remember that there are still several station areas within each station. Let's get all of the commuter indexes for each station area and then take the median of that commuter index to assign to the entire station. Create a sorted list of tuples in descending order containing the median commuter index and the station name.

In [1]:
#insert 33
station_dict = {}
for index, station in tuple_list:
    if station[2] in station_dict:
        station_dict[station[2]].append(index)
    else:
        station_dict[station[2]] = [index]
new_tuple_list = []
for station, indexes in station_dict.items():
    new_tuple_list.append((np.median(indexes), station))
new_tuple_list.sort(reverse = True)
print(new_tuple_list[:10])

NameError: name 'tuple_list' is not defined

### How can you use what you have done so far to make decisions about MTA advertising???

In [0]:
tuple_list = []
for station in bigDict:
    try:
        tuple_list.append((getCommuteIndex(station), station))
    except:
        print(station, 'is a problem')
tuple_list.sort(reverse = True)
print(tuple_list[:10])
new_tuples = {}
for count, station in tuple_list:
    if station[2] in new_tuples:
        new_tuples[station[2]].append(count)
    else:
        new_tuples[station[2]] = [count]
new_tuples_list = []
for station, counts in new_tuples.items():
    new_tuples_list.append((np.average(counts), station))
print(sorted(new_tuples_list, reverse = True)[:10])

In [0]:
def turnstileRiders(station, t1, t2):
    if t1 < station[0][0]:
        count1 = station[0][1]
    for time, count in station:
#         print(time)
        if t1 >= time:
            count1 = count
        if t2 >= time:
            count2 = count
    return count2 - count1


# def turnstileRiders(turnstileList,startTime,endTime):
#     ret = 0 #Return Value
#     n = len(turnstileList)
#     totalTime = endTime-startTime
#     #print("Total Time:",totalTime) #Debugging Purpose
#     for i in range(n-1): #A list of N items has (N-1) consecutive pairs
#         ti = turnstileList[i][0]
#         tf = turnstileList[i+1][0]
#         if((tf<startTime) or (ti>endTime)):
#             continue #Non-overlap Time Segment
#         #Overlapping Time
#         segmentTime = min(tf,endTime) - max(ti,startTime)
#         fullSegment = tf-ti
#         if(fullSegment==noDay): #Dealing with Bug
#             return 0
#         weight = segmentTime/fullSegment
#         peopleCount = turnstileList[i+1][1]-turnstileList[i][1]
#         peopleCount = cleanPeople(peopleCount)
#         #print("Considering",ti,"to",tf,"Weight:",weight,"Riders:",peopleCount) #Debugging Purpose
#         ret+= (weight)*peopleCount
#     return int(ret)


t1 = datetime.strptime("2017-06-12 00:00:00","%Y-%m-%d %H:%M:%S")
t2 = datetime.strptime("2017-06-13 00:00:00","%Y-%m-%d %H:%M:%S")
print(turnstileRiders(bigDict[('R204', 'R043', 'WALL ST')]["02-00-00"],t1,t2))

import datetime as dt
def getStationRangeRider(station, t1, t2):
    total = 0
    for scp, times_list in station.items():
        total += turnstileRiders(times_list, t1, t2)
    return total

print(getStationRangeRider(bigDict[('R204', 'R043', 'WALL ST')],dt.datetime(2017,6,12,0,0,0),dt.datetime(2017,6,13,0,0,0)))


def isWeekday(time):
    if time.weekday() <= 4:
        return True
    return False

def getCommuteIndex(station):
    t1 = dt.datetime(2017,6,9,0,0,0)
    weekday_exits = 0
    weekend_exits = 0
    weekdays = 0
    weekends = 0
    for i in range(7):
        t1 += dt.timedelta(days = 1)
        t2 = t1 + dt.timedelta(days = 1)
#         print(t1, getStationRangeRider(bigDict[station], t1, t2), isWeekday(t1))
#         print(t1, t2, isWeekday(t1))
        if isWeekday(t1):
#             print('adding to weekday_exits')
            weekday_exits += getStationRangeRider(bigDict[station], t1, t2)
            weekdays +=1
        else:
            weekend_exits += getStationRangeRider(bigDict[station], t1, t2)
            weekends+=1
#     print(weekdays, weekends)
    day = weekday_exits / 5
    end = weekend_exits / 2
    
#     print(day, end)
    if end == 0:
        return 0
    return weekday_exits / 5 - weekend_exits / 2

print(getCommuteIndex(('PTH11', 'R545', '14TH STREET')))

print("Lauren's numbers: 1419, 9507, 0.663")

isWeekday(dt.datetime(2017,6,12,0,0,0))

In [0]:
totals = []
counts_stations.sort(reverse = True)
for count, station in counts_stations[:5]:
    for each_one in range(count):
        totals.append(station)
plt.hist(totals, bins = np.arange(-0.5, 5.6, 1), rwidth = 0.5)
plt.xticks(fontsize = 7)
plt.yticks(fontsize = 8)
plt.xlabel('Station')
plt.ylabel('People')
plt.title('The most populous stations on 6/12/17')
plt.show()